<a href="https://colab.research.google.com/github/vedica1011/NLP_BERT_TL/blob/main/Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install datasets

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from datasets import Dataset

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
data = pd.read_csv("train_40k.csv",usecols=['Text','Cat2'])
data.columns = ['Text','label']
data.isnull().sum()

Text     0
label    0
dtype: int64

In [4]:
id2label = {idx:category for idx,category in enumerate(data['label'].unique())}
label2id = {v:k for k,v in id2label.items() }
data['label'] = data['label'].map(label2id)

In [5]:
dataset1 = Dataset.from_pandas(data)
final_data = dataset1.train_test_split(test_size=0.1)

In [6]:
final_data

DatasetDict({
    train: Dataset({
        features: ['Text', 'label'],
        num_rows: 36000
    })
    test: Dataset({
        features: ['Text', 'label'],
        num_rows: 4000
    })
})

In [7]:
def preprocess_function(examples):
    return tokenizer(examples["Text"], truncation=True,max_length=450)

In [8]:
tokenized_final_data = final_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [9]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [15]:
#!pip install evaluate

In [10]:
import evaluate
accuracy = evaluate.load("accuracy")

In [11]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [12]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_final_data["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [14]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=data['label'].nunique(), id2label=id2label,
    label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [200]:
type(tokenized_final_data["test"]['input_ids'][0])

list

In [201]:
tokenized_final_data["train"].features

{'Text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [202]:
tokenized_final_data["train"]

Dataset({
    features: ['Text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 36000
})

In [203]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_final_data["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_final_data["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [204]:
import tensorflow as tf
model.compile(optimizer=optimizer)

In [205]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [206]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="bert_classfier",
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/bert_classfier is already a clone of https://huggingface.co/vedica1011/bert_classfier. Make sure you pull the latest changes with `repo.git_pull()`.


In [207]:
callbacks = [metric_callback,push_to_hub_callback]

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)

Epoch 1/3
   2/2250 [..............................] - ETA: 19:04:51 - loss: 4.1424